In [9]:
# Deep Neural Network

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.examples.tutorials.mnist import input_data

def model_test():
    labels = sess.run(model,feed_dict={X:mnist.test.images,Y:mnist.test.labels,keep_prob:1})
    fig = plt.figure()
    for i in range(10):
        r = random.randrange(1,1000)
        subplot = fig.add_subplot(2,5,i+1)
        subplot.set_xticks([])
        subplot.set_yticks([])
        subplot.set_title("%d"%np.argmax(labels[r]))
        subplot.imshow(mnist.test.images[r].reshape((28,28)),cmap=plt.cm.gray_r)

    plt.show()

mnist = input_data.read_data_sets("C:\\Users\\k\\Documents\\Dataset\\MNIST",one_hot=True)
tf.reset_default_graph()
global_step = tf.Variable(0,trainable=False,name='global_step')

X = tf.placeholder(tf.float32,[None,784])
Y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32,name="dropout_rate")

with tf.name_scope('Layer1'):
    W1 = tf.Variable(tf.random_normal([784,256],stddev=0.01))
    b1 = tf.Variable(tf.zeros([256]))
    L1 = tf.add(tf.matmul(X,W1),b1)
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.dropout(L1, keep_prob)

with tf.name_scope('Layer2'):
    W2 = tf.Variable(tf.random_normal([256,256],stddev=0.01))
    b2 = tf.Variable(tf.zeros([256]))
    L2 = tf.add(tf.matmul(L1,W2),b2)
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.dropout(L2, keep_prob)
    
with tf.name_scope('Layer3'):
    W3 = tf.Variable(tf.random_normal([256,128],stddev=0.01))
    b3 = tf.Variable(tf.zeros([128]))
    L3 = tf.add(tf.matmul(L2,W3),b3)
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.dropout(L3, keep_prob)

with tf.name_scope('Output'):
    W4 = tf.Variable(tf.random_normal([128,10],stddev=0.01))
    b4 = tf.Variable(tf.zeros([10]))
    model = tf.add(tf.matmul(L3,W4),b4)
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=model))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(cost)
    
    tf.summary.scalar('cost',cost)
    
init = tf.global_variables_initializer()
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
forced = True

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path) and not forced:
    saver.restore(sess,ckpt.model_checkpoint_path)
    print("Already Trained !")
    model_test()
else:
    sess.run(init)

    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs',sess.graph)

    batch_size = 100
    total_batch = int(mnist.train.num_examples/batch_size)
    print('Global step : ',sess.run(global_step))
    for epoch in range(15):
        total_cost = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val = sess.run([train_op,cost],feed_dict={X:batch_xs,Y:batch_ys,keep_prob:0.8})
            total_cost += cost_val
            summary = sess.run(merged,feed_dict={X:batch_xs,Y:batch_ys,keep_prob:0.8})
            writer.add_summary(summary,global_step=sess.run(global_step))

        print('Epoch : {:04d}'.format(epoch+1))
        print('Avg. cost = {:.3f}'.format(total_cost/total_batch))
        

    saver.save(sess,'./model/dnn_mnist.ckpt',global_step=global_step)
    is_correct = tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))
    print("정확도 : {:.4f}".format(sess.run(accuracy,feed_dict={X:mnist.test.images,Y:mnist.test.labels,keep_prob:1})))




Extracting C:\Users\k\Documents\Dataset\MNIST\train-images-idx3-ubyte.gz
Extracting C:\Users\k\Documents\Dataset\MNIST\train-labels-idx1-ubyte.gz
Extracting C:\Users\k\Documents\Dataset\MNIST\t10k-images-idx3-ubyte.gz
Extracting C:\Users\k\Documents\Dataset\MNIST\t10k-labels-idx1-ubyte.gz
Global step :  0
Epoch : 0001
Avg. cost = 0.582
Epoch : 0002
Avg. cost = 0.194
Epoch : 0003
Avg. cost = 0.132
Epoch : 0004
Avg. cost = 0.106
Epoch : 0005
Avg. cost = 0.088
Epoch : 0006
Avg. cost = 0.076
Epoch : 0007
Avg. cost = 0.067
Epoch : 0008
Avg. cost = 0.060
Epoch : 0009
Avg. cost = 0.054
Epoch : 0010
Avg. cost = 0.048
Epoch : 0011
Avg. cost = 0.046
Epoch : 0012
Avg. cost = 0.043
Epoch : 0013
Avg. cost = 0.039
Epoch : 0014
Avg. cost = 0.036
Epoch : 0015
Avg. cost = 0.033
정확도 : 0.9799
